In [1]:
# !git clone https://github.com/open-mmlab/mmdetection.git

In [2]:
# from IPython.display import clear_output

# %cd /app/_data/mmdetection
# !pip install -r requirements/build.txt
# !pip install -v -e .  # or "python setup.py develop"
# !pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.1/index.html
# !pip install openmim
# !mim install mmdet
# clear_output()

In [3]:
import mmcv
import mmdet
from mmcv import Config
from mmdet.apis import (
    inference_detector,
    init_detector,
    set_random_seed,
    train_detector,
)
from mmdet.datasets import build_dataset
from mmdet.models import build_detector

In [4]:
SEED = 38
set_random_seed(SEED, deterministic=False)

In [5]:
import argparse
import ast
import glob
import json
import os
import random
import re
import shutil

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from IPython.core.interactiveshell import InteractiveShell
from PIL import Image
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

InteractiveShell.ast_node_interactivity = "all"
import seaborn as sns
import torch
import torchvision

In [7]:
TRAIN_DF_PART = "/app/_data/tensorflow-great-barrier-reef/train.csv"
IMAGE_FOLDER = "images"
LABEL_FOLDER = "labels"
DEVICE = "cuda" if torch.cuda.is_available else "cpu"
DEVICE

'cuda'

In [8]:
with open("/app/_data/sequences.json", "r") as f:
    seq_dict = json.load(f)

In [9]:
df = pd.read_csv(TRAIN_DF_PART)
df["img_path"] = (
    "/app/_data/tensorflow-great-barrier-reef/train_images/video_"
    + df.video_id.astype("str")
    + "/"
    + df.video_frame.astype("str")
    + ".jpg"
)
df["annotations"] = df["annotations"].apply(lambda x: ast.literal_eval(x))
df["len_annotation"] = df["annotations"].str.len()
df["image_id"] = df["image_id"].str.replace("-", "_", regex=True)
df["new_img_path"] = f"/app/_data/{IMAGE_FOLDER}/" + df["image_id"] + ".jpg"
df["label"] = df["len_annotation"].apply(lambda x: 0 if x == 0 else 1)
df["no_label"] = df["len_annotation"].apply(lambda x: True if x == 0 else False)
R = df[df["len_annotation"] == 0].shape[0] / df[df["len_annotation"] != 0].shape[0]
df["label_change"] = df["label"] & df["no_label"].shift(1) & df["no_label"].shift(
    2
) | df["no_label"] & df["label"].shift(1) & df["label"].shift(2)
df["sequense_change"] = df["sequence"] != df["sequence"].shift(1)
df["start_subseq"] = df["sequense_change"] | df["label_change"]
df.loc[df.index[-1], "start_subseq"] = True
df["start_subseq"].sum()
start_idx = 0
for subsequence_id, end_idx in enumerate(df[df["start_subseq"]].index):
    df.loc[start_idx:end_idx, "subsequence_id"] = subsequence_id
    start_idx = end_idx

df["subsequence_id"] = df["subsequence_id"].astype(int)
df["subsequence_id"].nunique()

138

137

# train_test_split

## random

In [10]:
l = 0
ll = 0
for i in tqdm(range(20000)):
    n_val = np.random.randint(13, 20, 1)[0]
    val_seq = random.sample(df.subsequence_id.unique().tolist(), n_val)
    train_seq = list(set(df.subsequence_id.tolist()) - set(val_seq))
    l = (
        df.query("subsequence_id in @train_seq")["label"].sum()
        / df.query("subsequence_id in @val_seq")["label"].sum()
    )
    ll = (
        df.query("subsequence_id in @train_seq")["len_annotation"].sum()
        / df.query("subsequence_id in @val_seq")["len_annotation"].sum()
    )
    r = (
        df.query("subsequence_id in @val_seq and len_annotation == 0").shape[0]
        / df.query("subsequence_id in @val_seq and len_annotation != 0").shape[0]
    )
    if 8 <= l <= 10 and 8 <= ll <= 10 and 3.5 <= r <= 4.3:
        break

df.query("subsequence_id in @train_seq")[["len_annotation", "label"]].sum() / df.query(
    "subsequence_id in @val_seq"
)[["len_annotation", "label"]].sum()

df.query("subsequence_id in @val_seq and len_annotation == 0").shape[0] / df.query(
    "subsequence_id in @val_seq and len_annotation != 0"
).shape[0]

sorted(val_seq)

 15% 2982/20000 [00:53<05:05, 55.79it/s]


len_annotation    8.442857
label             9.355789
dtype: float64

3.7873684210526317

[18, 20, 24, 31, 57, 63, 76, 77, 84, 86, 91, 97, 100, 110, 122, 134]

In [11]:
KFOLD = "rand"
train = pd.concat(
    [
        df.query("subsequence_id in @train_seq and len_annotation>0"),
        df.query("subsequence_id in @train_seq and len_annotation==0").sample(
            int(
                df.query("subsequence_id in @train_seq and len_annotation>0").shape[0]
                * 0.1
            )
        ),
    ]
)
val = df.query("subsequence_id in @val_seq")
train[["len_annotation", "label"]].sum()
val[["len_annotation", "label"]].sum()
train[["len_annotation", "label"]].sum() / val[["len_annotation", "label"]].sum()

len_annotation    10638
label              4444
dtype: int64

len_annotation    1260
label              475
dtype: int64

len_annotation    8.442857
label             9.355789
dtype: float64

In [12]:
%%writefile /app/_data/for_mmdet/labels.txt
starfish

Overwriting /app/_data/for_mmdet/labels.txt


In [13]:
def df_to_coco(df):
    images = []
    annotations = []
    img_id = 0
    annotation_id = 0
    categories = [
        {
            "id": 0,
            "name": "starfish",
        },
    ]
    for ix in tqdm(df.index.tolist()):
        img = Image.open(df.loc[ix, "img_path"])
        w, h = img.size
        bboxes = df.loc[ix, "annotations"]
        img_name = f'{df.loc[ix, "image_id"]}.jpg'
        img_dict = {
            "id": img_id,
            "file_name": img_name,
            "height": h,
            "width": w,
        }
        images.append(img_dict)
        if len(bboxes) != 0:
            for i in range(len(bboxes)):
                bbox = bboxes[i]
                bbox = [bbox["x"], bbox["y"], bbox["width"], bbox["height"]]
                annotations_dict = {
                    "id": annotation_id,
                    "image_id": img_id,
                    "category_id": 0,
                    "bbox": bbox,
                    "area": bbox[2] * bbox[3],
                    "segmentation": [],
                    "iscrowd": 0,
                }
                annotations.append(annotations_dict)
                annotation_id += 1
        img_id += 1
    coco_dict = {"categories": categories, "images": images, "annotations": annotations}
    return coco_dict

In [14]:
train_dict = df_to_coco(train)
with open("/app/_data/for_mmdet/coco_train.json", "w") as f:
    json.dump(train_dict, f, ensure_ascii=True, indent=4)
val_dict = df_to_coco(val)
with open("/app/_data/for_mmdet/coco_val.json", "w") as f:
    json.dump(val_dict, f, ensure_ascii=True, indent=4)

100% 4888/4888 [00:00<00:00, 5736.44it/s]
100% 2274/2274 [00:00<00:00, 6471.46it/s]


In [15]:
# val_dict

# Model Config

In [16]:
# baseline_cfg_path = "/kaggle/working/mmdetection/configs/cascade_rcnn/cascade_rcnn_x101_32x4d_fpn_1x_coco.py"
# cfg = Config.fromfile(baseline_cfg_path)
cfg = Config.fromfile(
    "/app/_data/mmdetection/configs/cascade_rcnn/cascade_rcnn_x101_32x4d_fpn_1x_coco.py"
)

In [17]:
model_name = "cascade_rcnn_x101_32x4d_fpn_1x_coco_cust1"
# Folder to store model logs and weight files
work_dir = f"/app/_data/for_mmdet/{model_name}"
cfg.work_dir = work_dir
if not os.path.exists(work_dir):
    os.makedirs(work_dir)

In [18]:
# cfg

Config (path: /app/_data/mmdetection/configs/cascade_rcnn/cascade_rcnn_x101_32x4d_fpn_1x_coco.py): {'model': {'type': 'CascadeRCNN', 'backbone': {'type': 'ResNeXt', 'depth': 101, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'norm_eval': True, 'style': 'pytorch', 'init_cfg': {'type': 'Pretrained', 'checkpoint': 'open-mmlab://resnext101_32x4d'}, 'groups': 32, 'base_width': 4}, 'neck': {'type': 'FPN', 'in_channels': [256, 512, 1024, 2048], 'out_channels': 256, 'num_outs': 5}, 'rpn_head': {'type': 'RPNHead', 'in_channels': 256, 'feat_channels': 256, 'anchor_generator': {'type': 'AnchorGenerator', 'scales': [8], 'ratios': [0.5, 1.0, 2.0], 'strides': [4, 8, 16, 32, 64]}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [1.0, 1.0, 1.0, 1.0]}, 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0}, 'loss_bbox': {'type': 'SmoothL1Loss', 'beta': 0.11

In [19]:
for head in cfg.model.roi_head.bbox_head:
    head.num_classes = 1

In [20]:
cfg.runner.max_epochs = 20

In [21]:
cfg.optimizer.lr = 0.02 / 8

In [22]:
cfg.lr_config

{'policy': 'step',
 'warmup': 'linear',
 'warmup_iters': 500,
 'warmup_ratio': 0.001,
 'step': [8, 11]}

In [23]:
cfg.lr_config = dict(
    policy='CosineAnnealing', # The policy of scheduler, also support CosineAnnealing, Cyclic, etc. Refer to details of supported LrUpdater from https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/lr_updater.py#L9.
    by_epoch=False,
    warmup='linear', # The warmup policy, also support `exp` and `constant`.
    warmup_iters=500, # The number of iterations for warmup
    warmup_ratio=0.001, # The ratio of the starting learning rate used for warmup
    min_lr=1e-07)

In [25]:
cfg.log_config.interval = 100

In [26]:
cfg.data.samples_per_gpu=4
cfg.data.workers_per_gpu=4

In [27]:
cfg.dataset_type

'CocoDataset'

In [28]:
cfg.gpu_ids = [0]
cfg.classes = '/app/_data/for_mmdet/labels.txt'

In [29]:
cfg.data.train.ann_file =  "/app/_data/for_mmdet/coco_train.json"
cfg.data.train.img_prefix =  "/app/_data/images/"
cfg.data.train.classes = cfg.classes

In [30]:
cfg.data.val.ann_file =  "/app/_data/for_mmdet/coco_val.json"
cfg.data.val.img_prefix =  "/app/_data/images/"
cfg.data.val.classes = cfg.classes

In [31]:
cfg.data.test = cfg.data.val

In [32]:
cfg.seed = SEED

In [33]:
cfg.evaluation

{'interval': 1, 'metric': 'bbox'}

In [34]:
import wandb
wandb.login()

wandb: Currently logged in as: tatanko (use `wandb login --relogin` to force relogin)


True

In [35]:
cfg.log_config.hooks

[{'type': 'TextLoggerHook'}]

In [36]:
cfg.log_config.hooks = [dict(type='TextLoggerHook'),
                        dict(type='WandbLoggerHook',
                             init_kwargs=dict(project='cascade_101',
                                              name=f'exp-{model_name}',
                                              entity='tatanko'))]

In [37]:
cfg.train_pipeline 

[{'type': 'LoadImageFromFile'},
 {'type': 'LoadAnnotations', 'with_bbox': True},
 {'type': 'Resize', 'img_scale': (1333, 800), 'keep_ratio': True},
 {'type': 'RandomFlip', 'flip_ratio': 0.5},
 {'type': 'Normalize',
  'mean': [123.675, 116.28, 103.53],
  'std': [58.395, 57.12, 57.375],
  'to_rgb': True},
 {'type': 'Pad', 'size_divisor': 32},
 {'type': 'DefaultFormatBundle'},
 {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels']}]

In [38]:
albu_train_transforms = [
    dict(type='ShiftScaleRotate', shift_limit=0.0625,
         scale_limit=0.15, rotate_limit=15, p=0.05),
    dict(type='RandomBrightnessContrast', brightness_limit=0.2,
         contrast_limit=0.2, p=0.3),
    dict(type='IAAAffine', shear=(-10.0, 10.0), p=0.4),
    dict(type="Blur", p=1.0, blur_limit=7),
    dict(type='CLAHE', p=0.5),
    dict(type='Equalize', mode='cv', p=0.4),
    dict(
        type="OneOf",
        transforms=[
            dict(type="GaussianBlur", p=1.0, blur_limit=7),
            dict(type="MedianBlur", p=1.0, blur_limit=7),
        ],
        p=0.4,
    ),
    
    dict(type='MixUp', p=0.2, lambd=0.5),
    dict(type='Flip', p=0.5),
    ]


cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=False),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
        type='BboxParams',
        format='pascal_voc',
        label_fields=['gt_labels'],
        min_visibility=0.5,
        filter_lost_elements=True),
        keymap=dict(img='image', gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks'])
]
cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]

In [39]:
model = build_detector(
    cfg.model, train_cfg=cfg.get("train_cfg"), test_cfg=cfg.get("test_cfg")
)
model.init_weights()

2022-02-10 14:08:15,659 - mmcv - INFO - initialize ResNeXt with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://resnext101_32x4d'}
2022-02-10 14:08:15,660 - mmcv - INFO - load model from: open-mmlab://resnext101_32x4d
2022-02-10 14:08:15,661 - mmcv - INFO - load checkpoint from openmmlab path: open-mmlab://resnext101_32x4d
2022-02-10 14:08:15,878 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-02-10 14:08:15,900 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-02-10 14:08:15,905 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-02-10 14:08:16,001 - mmcv - INFO - initialize Shared2FCBBo

In [40]:
train_dataset = build_dataset(cfg.data.train)

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!


In [41]:
train_dataset


CocoDataset Train dataset with number of images 4444, and instance counts: 
+--------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+
| category     | count | category | count | category | count | category | count | category | count |
+--------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+
|              |       |          |       |          |       |          |       |          |       |
| 0 [starfish] | 10638 |          |       |          |       |          |       |          |       |
+--------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+

In [42]:
train_detector(model, train_dataset, cfg, distributed=False, validate=True)

2022-02-10 14:08:25,358 - mmdet - INFO - Start running, host: root@13ada95134ff, work_dir: /app/_data/for_mmdet/cascade_rcnn_x101_32x4d_fpn_1x_coco_cust1
2022-02-10 14:08:25,359 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(LOW         ) IterTimerHook                     

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2022-02-10 14:09:53,294 - mmdet - INFO - Epoch [1][100/1111]	lr: 4.970e-04, eta: 5:12:17, time: 0.847, data_time: 0.036, memory: 12463, loss_rpn_cls: 0.3419, loss_rpn_bbox: 0.0166, s0.loss_cls: 0.1121, s0.acc: 98.6992, s0.loss_bbox: 0.0071, s1.loss_cls: 0.0557, s1.acc: 99.3257, s1.loss_bbox: 0.0021, s2.loss_cls: 0.0356, s2.acc: 95.2915, s2.loss_bbox: 0.0002, loss: 0.5714
2022-02-10 14:11:14,449 - mmdet - INFO - Epoch [1][200/1111]	lr: 9.963e-04, eta: 5:04:21, time: 0.812, data_time: 0.012, memory: 12463, loss_rpn_cls: 0.1121, loss_rpn_bbox: 0.0125, s0.loss_cls: 0.0790, s0.acc: 98.3530, s0.loss_bbox: 0.0259, s1.loss_cls: 0.0270, s1.acc: 99.0112, s1.loss_bbox: 0.0098, s2.loss_cls: 0.0104, s2.acc: 99.3315, s2.loss_bbox: 0.0019, loss: 0.2785
2022-02-10 14:12:34,781 - mmdet - INFO - Epoch [1][300/1111]	lr: 1.495e-03, eta: 4:59:48, time: 0.803, data_time: 0.012, memory: 12463, loss_rpn_cls: 0.0849, loss_rpn_bbox: 0.0124, s0.loss_cls: 0.1013, s0.acc: 97.0796, s0.loss_bbox: 0.0644, s1.loss_cls

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 2274/2274, 13.0 task/s, elapsed: 174s, ETA:     0s

2022-02-10 14:26:22,604 - mmdet - INFO - Evaluating bbox...


...

2022-02-10 20:05:35,086 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


2022-02-10 20:05:35,532 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.160
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.367
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.106
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.198
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.209
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.209
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.209
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.006
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.251
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100

DONE (t=0.38s).
Accumulating evaluation results...
DONE (t=0.06s).


learning_rate,▄██████▇▇▇▇▇▆▆▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,▅████▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_bbox,█▆▄▄▄▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_cls,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/s0.acc,▆▁▂▃▂▄▄▄▅▅▅▅▅▆▆▆▇▆▇▇▇▇▇▇▇▇█▇████████████
train/s0.loss_bbox,▃█▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/s0.loss_cls,▆█▆▆▆▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/s1.acc,▇▁▂▃▃▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇████████████████
train/s1.loss_bbox,▁▆███▇▇▆▆▆▅▅▅▅▄▄▃▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
train/s1.loss_cls,▄█▇▆▆▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
